In [1]:
import numpy as np
import pandas as pd

data = pd.read_csv('data.csv')

In [2]:
#Full list without zero scores
#no_zero_score_data = data.loc[data['score'] > 0]

#List with zero scores and a status of 1, 2, or 6 (see MAL-Scraper)
#status = {1, 2, 6}
#WCPtW_data = data.loc[data['status'].isin(status)]

#List with zero scores and a status of 1 or 2
#WC_data = data.loc[data['status'] <= 2]

#List without zero scores and a status of 2
C_data = data.loc[(data['status'] == 2) & (data['score'] > 0)]

#C_train = C_data.sample(frac=0.8, random_state=1)
#C_test = C_data.drop(C_train.index)


In [3]:
from lightfm.data import Dataset

#Building the dataset
dataset = Dataset()
dataset.fit(C_data['user'].values, C_data['anime_id'].values)

#Building the interactions matrix and the user features matrix
(interactions, weights) = dataset.build_interactions(zip(C_data['user'].values,
                                                         C_data['anime_id'].values,
                                                         C_data['score'].values))

E:\Anaconda\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [4]:
from lightfm import LightFM

model = LightFM(loss='bpr')
model.fit(interactions, sample_weight=weights, epochs=10)

In [34]:
mappings = dataset.mapping()
# Dict of the form {username : LightFM userid}
users = mappings[0]
# Dict of the form {LightFM animeid : MAL animeid}
animes = {v: k for k, v in mappings[2].items()}
animes_list = np.array([*animes.keys()])

In [74]:
from operator import itemgetter
user = "FinalReality56"   

user_id = 0 #users[user]

known_positives = C_data.loc[C_data['user'] == user]
known_positives = known_positives['anime_id'].values
for i in range(len(known_positives)):
    known_positives[i] = mappings[2][known_positives[i]]

unwatched_anime = np.setxor1d(known_positives, animes_list)

scores = model.predict(user_id, unwatched_anime)
scores = list(zip(unwatched_anime, scores))
scores.sort(key=itemgetter(1))

n = 10
topn = [scores[i][0] for i in range(n)]
top10 = [animes[i] for i in topn]

[28891, 32935, 20583, 24415, 16894, 11771, 21995, 23273, 36949, 35760]
